In [3]:

!pip install langchain langchain-community pypdf sentence-transformers faiss-cpu transformers pillow torch

In [6]:
# Quick start - just copy and run this
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# 1. Setup PDF RAG
loader = PyPDFLoader("/content/BNBC 2020_Earthquake load.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 2. Setup image model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("✅ System ready!\n")

# 3. Function for text queries
def ask_text(question):
    docs = vectorstore.similarity_search(question, k=3)
    print(f"\nQ: {question}\n")
    for i, doc in enumerate(docs, 1):
        print(f"[{i}] Page {doc.metadata.get('page')}:\n{doc.page_content}\n")
    return docs

# 4. Function for image queries
def ask_image(image_path, question):
    # Analyze image
    img = Image.open(image_path).convert('RGB')
    inputs = processor(img, "construction detail", return_tensors="pt")
    out = blip_model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    print(f"\nImage: {caption}")
    print(f"Q: {question}\n")

    # Search with enhanced query
    enhanced = f"{question}. Image shows: {caption}"
    docs = vectorstore.similarity_search(enhanced, k=3)

    for i, doc in enumerate(docs, 1):
        print(f"[{i}] Page {doc.metadata.get('page')}:\n{doc.page_content}\n")
    return docs

# 5. Use it!
ask_text("What is the seismic zone for Dhaka?")

# Upload image first, then:
# ask_image("/content/your_image.jpg", "Is this compliant?")

/tmp/ipython-input-1225512831.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/473 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

✅ System ready!


Q: What is the seismic zone for Dhaka?

[1] Page 10:
evsjv‡`k †M‡RU, AwZwi³, †deªæqvwi 11, 2021 3195    
 
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Figure 6.2.24 Seismic zoning map of Bangladesh

[2] Page 11:
3196 evsjv‡`k †M‡RU, AwZwi³, †deªæqvwi 11, 2021  
 
Table 6.2.14: Description of Seismic Zones  
Seismic 
Zone  Location  Seismic 
Intensity  
Seismic  Zone 
Coefficient, Z 
1 Southwestern part including Barisal, Khulna, 
J essore, Rajshahi 
Low  0.12  
2 Lower Central and Northwestern part including 
Noakhali, Dhaka, Pabna, Dinajpur, as well as 
Southwestern corner including Sundarbans  
Moderate  0.20  
3 U pper Central and Northwestern part including 
Brahmanbaria, Sirajganj, Rangpur  
Severe  0.28  
4 Northeastern part including Sylhet, 
Mymensingh, Kurigram  
Very  Severe  0.36  
Table 6.2.15:  Seismic Zone Coefficient Z  for Some Important Towns of Bangladesh   
Town  Z Town  Z Town  Z Town  Z 
Bagerhat 0.12  Gaibandha 

[Document(id='dfeaae50-af05-4e95-8acb-fc2b4ac533f3', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-05T11:37:29+00:00', 'source': '/content/BNBC 2020_Earthquake load.pdf', 'total_pages': 64, 'page': 10, 'page_label': '11'}, page_content='evsjv‡`k †M‡RU, AwZwi³, †deªæqvwi 11, 2021 3195    \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nFigure 6.2.24 Seismic zoning map of Bangladesh'),
 Document(id='6ec7dfce-dc8a-417a-bd8c-2db15d44f921', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-05T11:37:29+00:00', 'source': '/content/BNBC 2020_Earthquake load.pdf', 'total_pages': 64, 'page': 11, 'page_label': '12'}, page_content='3196 evsjv‡`k †M‡RU, AwZwi³, †deªæqvwi 11, 2021  \n \nTable 6.2.14: Description of Seismic Zones  \nSeismic \nZone  Location  Seismic \nIntensity  \nSeismic  Zone \nCoefficient, Z \n1 Southwestern part incl

In [8]:
from google.colab import files

# Upload image
print("Upload your building photo:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Query it
ask_image(image_path, "Does this meet BNBC 2020 requirements?")

Upload your building photo:


Saving EQ 2.jfif to EQ 2.jfif

Image: construction detail of the building
Q: Does this meet BNBC 2020 requirements?

[1] Page 62:
away from the centre of the building as feasible ar e to be designed 
exclusively for 1.5 times the lateral shear force calculated before.  
2.5.18  Non-Building Structures 
Calculation of seismic design forces on non-buildin g structures (e.g. chimney, self-
supported overhead water/fluid tank, silo, trussed tower, storage tank, cooling tower, 
monument and other structures not covered in Sec 2. 5) shall be in accordance with 
"Chapter 15: Seismic Design Requirements for Non-Bu ilding Structures, Minimum 
Design Loads for Buildings and Other Structures, AS CE Standard ASCE/SEI 7-05" 
complying with the requirements of Sec 2.5 of this Code.

[2] Page 45:
that are permanently attached to structures and for their supports and 
attachments. The following components are exempt from the requirements of 
this Section. 
(1)  Architectural components in Seismic Desi

[Document(id='19b05086-6b34-4c35-a356-336b5d6eb1c7', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-05T11:37:29+00:00', 'source': '/content/BNBC 2020_Earthquake load.pdf', 'total_pages': 64, 'page': 62, 'page_label': '63'}, page_content='away from the centre of the building as feasible ar e to be designed \nexclusively for 1.5 times the lateral shear force calculated before.  \n2.5.18  Non-Building Structures \nCalculation of seismic design forces on non-buildin g structures (e.g. chimney, self-\nsupported overhead water/fluid tank, silo, trussed tower, storage tank, cooling tower, \nmonument and other structures not covered in Sec 2. 5) shall be in accordance with \n"Chapter 15: Seismic Design Requirements for Non-Bu ilding Structures, Minimum \nDesign Loads for Buildings and Other Structures, AS CE Standard ASCE/SEI 7-05" \ncomplying with the requirements of Sec 2.5 of this Code.'),
 Document(id='5cc1d581-26df-425e-abfd-5ee63ab1392f', me

In [9]:
from google.colab import files

# Upload image
print("Upload your building photo:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Query it
ask_image(image_path, "Does this meet BNBC 2020 requirements?")

Upload your building photo:


Saving EQ 1.jfif to EQ 1.jfif

Image: construction detail of the new delhi metro station
Q: Does this meet BNBC 2020 requirements?

[1] Page 45:
that are permanently attached to structures and for their supports and 
attachments. The following components are exempt from the requirements of 
this Section. 
(1)  Architectural components in Seismic Design Category B, other than 
parapets supported by bearing walls or shear walls, where the 
component importance factor, .+ is equal to 1.0. 
(2)  Mechanical and electrical components in Seismic Design Category B. 
(3)  Mechanical and electrical components in Seismic Design Category C 
where the importance factor, .+ is equal to 1.0. 
(4)  Mechanical and electrical components in Seismic Design Category D 
where the component importance factor, .+ is equal to 1.0 and either 
(a) flexible connections between the components and associated 
ductwork, piping, and conduit are provided, or (b) components are 
mounted at 1.2 m or less above a floor l

[Document(id='5cc1d581-26df-425e-abfd-5ee63ab1392f', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-05T11:37:29+00:00', 'source': '/content/BNBC 2020_Earthquake load.pdf', 'total_pages': 64, 'page': 45, 'page_label': '46'}, page_content='that are permanently attached to structures and for their supports and \nattachments. The following components are exempt from the requirements of \nthis Section. \n(1)  Architectural components in Seismic Design Category B, other than \nparapets supported by bearing walls or shear walls, where the \ncomponent importance factor, .+ is equal to 1.0. \n(2)  Mechanical and electrical components in Seismic Design Category B. \n(3)  Mechanical and electrical components in Seismic Design Category C \nwhere the importance factor, .+ is equal to 1.0. \n(4)  Mechanical and electrical components in Seismic Design Category D \nwhere the component importance factor, .+ is equal to 1.0 and either \n(a) flexible connect

In [11]:
from google.colab import files

# Upload image
print("Upload your building photo:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Query it
ask_image(image_path, "What are the seismic design category requirements for this construction detail according to BNBC 2020?")

Upload your building photo:


Saving EQ 3.jfif to EQ 3 (1).jfif

Image: construction detail of a building
Q: What are the seismic design category requirements for this construction detail according to BNBC 2020?

[1] Page 45:
that are permanently attached to structures and for their supports and 
attachments. The following components are exempt from the requirements of 
this Section. 
(1)  Architectural components in Seismic Design Category B, other than 
parapets supported by bearing walls or shear walls, where the 
component importance factor, .+ is equal to 1.0. 
(2)  Mechanical and electrical components in Seismic Design Category B. 
(3)  Mechanical and electrical components in Seismic Design Category C 
where the importance factor, .+ is equal to 1.0. 
(4)  Mechanical and electrical components in Seismic Design Category D 
where the component importance factor, .+ is equal to 1.0 and either 
(a) flexible connections between the components and associated 
ductwork, piping, and conduit are provided, or (b) compo

[Document(id='5cc1d581-26df-425e-abfd-5ee63ab1392f', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2026-02-05T11:37:29+00:00', 'source': '/content/BNBC 2020_Earthquake load.pdf', 'total_pages': 64, 'page': 45, 'page_label': '46'}, page_content='that are permanently attached to structures and for their supports and \nattachments. The following components are exempt from the requirements of \nthis Section. \n(1)  Architectural components in Seismic Design Category B, other than \nparapets supported by bearing walls or shear walls, where the \ncomponent importance factor, .+ is equal to 1.0. \n(2)  Mechanical and electrical components in Seismic Design Category B. \n(3)  Mechanical and electrical components in Seismic Design Category C \nwhere the importance factor, .+ is equal to 1.0. \n(4)  Mechanical and electrical components in Seismic Design Category D \nwhere the component importance factor, .+ is equal to 1.0 and either \n(a) flexible connect

In [ ]:
from google.colab import files

# Upload image
print("Upload your building photo:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Query it
ask_image(image_path, "Can you identify the seismic zone, local site conditions, and importance class relevant to this construction detail, as per BNBC 2020, to determine its seismic design category?")

Upload your building photo:
